In [8]:
import pandas as pd
import os

carpeta = "Data encontrada"

# Lista para almacenar cada DataFrame
dataframes = []

#dff = pd.read_excel('Data encontrada/resultados_inalco.xlsx')

# Iterar sobre todos los archivos en la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith(".xlsx") or archivo.endswith(".xls"):
        ruta = os.path.join(carpeta, archivo)
        print(f"Leyendo: {archivo}")
        df = pd.read_excel(ruta)
        nombre_archivo = os.path.splitext(archivo)[0]  # Quita la extensión
        df["Pagina"] = nombre_archivo.replace("resultados_", "")
        df["Pagina"] = df["Pagina"].str.upper()
        dataframes.append(df)

df_concatenado = pd.concat(dataframes, ignore_index=True)


Leyendo: resultados_autoplanet.xlsx
Leyendo: resultados_casaderepuestos.xlsx
Leyendo: resultados_chilerepuestos.xlsx
Leyendo: resultados_ciper.xlsx
Leyendo: resultados_emgi.xlsx
Leyendo: resultados_inalco.xlsx
Leyendo: resultados_mundorepuestos.xlsx
Leyendo: resultados_salfarepuestos.xlsx
Leyendo: resultados_takora.xlsx
Leyendo: resultados_ulti.xlsx


LIMPIEZA DE DATOS

In [9]:
df_concatenado = df_concatenado.drop_duplicates(subset=[ 'Link']) #'Nombre Producto',


df_concatenado['Precio'] = (
    df_concatenado['Precio']
    .astype(str)
    .str.replace('$', '', regex=False)
    .str.replace('.', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.strip()
)

df_concatenado['Precio'] = pd.to_numeric(df_concatenado['Precio'], errors='coerce')

df_precios_cero = df_concatenado[df_concatenado['Precio'].isna() | (df_concatenado['Precio'] == 0)].copy()
df_concatenado = df_concatenado[~df_concatenado.index.isin(df_precios_cero.index)].copy()


mask_dup = df_concatenado.duplicated(subset=['Nombre Producto'], keep=False)
df_nombres_duplicados = df_concatenado[mask_dup].copy()
df_concatenado = df_concatenado[~mask_dup].copy()

conteos = df_nombres_duplicados['Pagina'].value_counts()
print(conteos)



# df_nombres_repetidos = df_nombres_duplicados[~df_nombres_duplicados["Pagina"].isin(["MUNDOREPUESTOS","CHILEREPUESTOS","CIPER",])]
# df_nombres_duplicados = df_nombres_duplicados[df_nombres_duplicados["Pagina"].isin(["MUNDOREPUESTOS","CHILEREPUESTOS","CIPER",])] 

Pagina
CHILEREPUESTOS     4739
MUNDOREPUESTOS     4251
CIPER              1564
TAKORA              636
INALCO              303
CASADEREPUESTOS     194
AUTOPLANET          130
EMGI                 40
Name: count, dtype: int64


GUARDANDO ARCHIVOS

In [31]:
def contiene_marca_modelo(row):
    nombre = str(row["Nombre Producto"]).lower()
    marca = str(row["Marca Buscada"]).lower()
    modelo = str(row["Modelo Buscado"]).lower()
    return marca in nombre and modelo in nombre

df_valido = df_concatenado[df_concatenado.apply(contiene_marca_modelo, axis=1)].copy()

df_repuestos = pd.read_csv('Input Repuestos/input_repuestos.csv')
lista_repuestos = df_repuestos['repuestos'].dropna().tolist()

lista_repuestos = [str(r).strip().lower() for r in lista_repuestos]

def detectar_tipo_repuesto(nombre_producto):
    nombre = str(nombre_producto).lower()
    for repuesto in lista_repuestos:
        if repuesto in nombre:
            return repuesto
    return None

# Aplicar la función
df_valido['tipo_repuesto'] = df_valido['Nombre Producto'].apply(detectar_tipo_repuesto)

df_con_tipo = df_valido[df_valido['tipo_repuesto'].notna()].copy()


columnas_necesarias = ['Nombre Producto', 'Marca Buscada', 'Modelo Buscado','tipo_repuesto', 'Precio', 'Pagina', 'Link', 'Imagen', 'Descripción', 'fecha_carga']
df_con_tipo = df_con_tipo[columnas_necesarias].copy()

# Paso 2: Renombrar columnas
nuevos_nombres = {
    'Nombre Producto': 'nombre',
    'Marca Buscada': 'marca',
    'Modelo Buscado': 'modelo',
    'Precio': 'precio',
    'Pagina': 'pagina',
    'Link': 'link',
    'Imagen': 'imagen',
    'Descripción': 'descripcion',
    'fecha_carga' : 'fecha_carga'
}
df_con_tipo.rename(columns=nuevos_nombres, inplace=True)
df_con_tipo.to_excel('data_carga.xlsx', index=False)

